In [23]:
import numpy as np 
import pandas as pd
import copy
import sys
from astropy.io import fits
import plotly.graph_objects as go
from astropy.coordinates import SkyCoord, GalacticLSR, Galactic
import astropy.units as u
sys.path.append('../../dash_gaia_orbits/')

# Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

In [24]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

dfme = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
dfme = dfme.drop(columns=['x', 'y', 'z'])
dfme = dfme[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars', 'mass_50', 'n_sne_50']]
dfme = dfme.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

In [25]:
dfgamma_ivs = dfme.loc[dfme['name'].isin(['Pozzo_1', 'CWNU_1083', 'CWNU_1096', 'OC_0470', 'OC_0479'])]
dfgamma = dfme.loc[dfme['family'] == 'gamma_velorum']
dfcr135 = dfme.loc[dfme['name'].isin(['Alessi_36', 'Collinder_135', 'Collinder_140', 'NGC_2451B', 'OCSN_82'])] 
dft10 = dfme.loc[dfme['name'].isin(['Trumpler_10'])]
dfsun = pd.DataFrame({'name' : ['Sun'], 'age_myr' : [100000], 'U' : [0], 'V' : [0], 'W' : [0], 'x' : [0], 'y' : [0], 'z' : [0], 'n_stars' : [1]})

dfcr135_full = dfme.loc[dfme['family'] == 'cr135']
dfm6_full = dfme.loc[dfme['family'] == 'm6']

In [26]:
# cr135_sc = SkyCoord(
#     ra = dfcr135['ra'].values * u.deg,
#     dec = dfcr135['dec'].values * u.deg,
#     distance = dfcr135['distance_50'].values * u.pc,
#     pm_ra_cosdec = dfcr135['pmra'].values * u.mas/u.yr,
#     pm_dec = dfcr135['pmdec'].values * u.mas/u.yr,
#     radial_velocity = dfcr135['radial_velocity'].values * u.km/u.s
# )
# cr135_sc = cr135_sc.transform_to(GalacticLSR)
# cr135_sc.representation_type = 'cartesian'
# cr135_sc.differential_type = 'cartesian'

# uvw = cr135_sc.velocity.d_xyz.value
# dfcr135[['U', 'V', 'W']] = uvw.T

# dfcr135 = dfcr135[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars', 'mass_50']]
# dfgamma = dfgamma[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars', 'mass_50']]
# dft10 = dft10[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars', 'mass_50']]
# dfsun = dfsun[['name', 'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars']]
# dfcr135_full = dfcr135_full[['name',  'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars', 'mass_50']]
# dfm6_full = dfm6_full[['name',  'age_myr', 'U', 'V', 'W', 'x', 'y', 'z', 'n_stars', 'mass_50']]

In [27]:
M_stars = dfgamma['mass_50'].sum()
M_gas = (M_stars/.02)
M_gas/1e4

9.433106431411565

In [28]:
cr135 = StarClusterData(dfcr135, data_name = 'Cr135 family (IVS)', min_size = 3, max_size = 12, color = 'orange', opacity = 1, marker_style = 'circle', show_tracks = True)
# gamma = StarClusterData(dfgamma, data_name = 'Gamma Vel family (IVS)', min_size = 3, max_size = 12, color = 'red', opacity = 1, marker_style = 'circle', show_tracks = True)
gamma_ivs = StarClusterData(dfgamma_ivs, data_name = 'Gamma Vel family (IVS)', min_size = 3, max_size = 12, color = 'red', opacity = .5, marker_style = 'circle', show_tracks = True)
t10 = StarClusterData(dft10, data_name = 'Trumpler 10', min_size = 3, max_size = 12, color = 'cyan', opacity = 1, marker_style = 'circle', show_tracks = True)
# cr135_full = StarClusterData(dfcr135_full, data_name = 'Cr135 family (full)', min_size = 3, max_size = 12, color = 'orange', opacity = .2, marker_style = 'circle', show_tracks = False)
# m6_full = StarClusterData(dfm6_full, data_name = 'M6 family (full)', min_size = 3, max_size = 12, color = 'cyan', opacity = .2, marker_style = 'circle', show_tracks = False)
# sun = StarClusterData(dfsun, data_name = 'Sun', min_size = 0, max_size = 7, color = 'yellow', opacity = 1, marker_style = 'circle', show_tracks = False)

# sc = StarClusterCollection([cr135, gamma, t10, cr135_full, m6_full, sun])

In [29]:
# read in Annie's paths
#path_annie = '/Users/cam/Downloads/all_paths/'
annie_data = pd.read_csv('/Users/cam/Downloads/all_cluster_info.csv')
annie_data.rename(columns={'age[Myr]' : 'age_myr', 'x_helio[pc]' : 'x', 'y_helio[pc]' : 'y', 'z_helio[pc]' : 'z', 'U_helio[km/s]' : 'U', 'V_helio[km/s]' : 'V', 'W_helio[km/s]' : 'W', 'Name':'name'}, inplace=True)

stars = annie_data.loc[annie_data['name'].isin(['zeta pup', 'Gamma2 Vel'])]
#clusters = annie_data.loc[~annie_data['name'].isin(['zeta pup', 'Gamma2 Vel'])]

# cr135_clusters = annie_data.loc[annie_data['name'].isin(['Alessi_36', 'Collinder_135', 'NGC_2451B', 'OC_0450'])]
# gamma_vel_clusters = annie_data.loc[annie_data['name'].isin(['CWNU_1083', 'CWNU_1096', 'OC_0470', 'OC_0479', 'Pozzo_1'])]
# t10_cluster = annie_data.loc[annie_data['name'].isin(['Trumpler_10'])]
cr135_clusters = dfme.loc[dfme['name'].isin(['Alessi_36', 'Collinder_135', 'Collinder_140', 'NGC_2451B', 'OC_0450', 'OCSN_82'])]
gamma_vel_clusters = dfme.loc[dfme['name'].isin(['CWNU_1083', 'CWNU_1096', 'OC_0470', 'OC_0479', 'Pozzo_1'])]
t10_cluster = dfme.loc[dfme['name'].isin(['Trumpler_10'])]


stars = StarClusterData(stars, data_name = 'Stars', min_size = 3, max_size = 12, color = 'forestgreen', opacity = 1, marker_style = 'circle', show_tracks = True)
cr135 = StarClusterData(cr135_clusters, data_name = 'Cr135 family', min_size = 3, max_size = 12, color = 'purple', opacity = 1, marker_style = 'circle', show_tracks = True)
gamma_vel = StarClusterData(gamma_vel_clusters, data_name = 'Gamma Vel family', min_size = 3, max_size = 12, color = 'crimson', opacity = 1, marker_style = 'circle', show_tracks = True)
t10 = StarClusterData(t10_cluster, data_name = 'Trumpler 10', min_size = 3, max_size = 12, color = 'darkcyan', opacity = 1, marker_style = 'circle', show_tracks = True)
sc = StarClusterCollection([cr135, gamma_vel, t10, stars])

In [30]:
# in LSR frame

# Create traces
static_traces = []
static_trace_times = []


time_show_ivs = np.round(np.arange(0., -4., -0.5), 1)

ivs = pd.read_csv('/Users/cam/Downloads/npix128boundary_mean_sigma10.csv')
ivs = pd.read_csv('/Users/cam/Downloads/peaks_meanmap_xyz.csv')
#ivs_out = pd.read_csv('/Users/cam/Downloads/npix128boundaryout_mean_sigma10.csv')
#ivs = pd.concat([ivs, ivs_out])
ivs = ivs.sample(frac=.7)
scatter_ivs = go.Scatter3d(
    x = ivs['x[pc]'].values,
    y = ivs['y[pc]'].values,
    z = ivs['z[pc]'].values,
    mode='markers',
    marker=dict(size=.3,
                color='dodgerblue',
                symbol='circle',
                opacity=.3),
    name='IVS',
    visible = True,
    hoverinfo='skip'
    )
static_traces.append(scatter_ivs)
static_trace_times.append([0])


scatter_sun_dir = go.Scatter3d(
    x = [-44, 0],
    y = [-242, 0],
    z = [-47, 0],
    mode='lines',
    line=dict(color='yellow', width=10),
    name='Sun direction',
    visible = 'legendonly',
    hoverinfo='skip'
    )
static_traces.append(scatter_sun_dir)
static_trace_times.append([0])

# annotate_sun_dir = go.Scatter3d(
#     x = [-40],
#     y = [-242],
#     z = [-20],
#     mode='text',
#     text=['Towards Sun'],
#     #textposition='top center',
#     textfont=dict(size=12, color='black'),
#     name='Sun direction',
#     visible = True,
#     hovertext='Sun direction',
#     hoverinfo='skip'
#     )
# static_traces.append(annotate_sun_dir)
# static_trace_times.append([0])


In [31]:
time_int = np.round(np.arange(0., -30.5, -0.5),1)

In [32]:
# #xyz_widths = (500, 500, 300)
# #xyz_ranges = ([-300, 300], [-800, -200], [-300, 300])
# xyz_ranges = (
#     [np.nanmedian(ivs['x[pc]']) - 201, np.nanmedian(ivs['x[pc]']) + 201],
#     [np.nanmedian(ivs['y[pc]']) - 201, np.nanmedian(ivs['y[pc]']) + 201],
#     [np.nanmedian(ivs['z[pc]']) - 150, np.nanmedian(ivs['z[pc]']) + 150]
# )

# plot_3d = StarClusters3DPlotter(
#     data_collection = sc, 
#     #xyz_widths = xyz_widths, 
#     figure_title='Interactive Figure 8 from Gao et al. (2024)',
#     xyz_ranges=xyz_ranges,
#     figure_theme = 'light', 
#     #trace_grouping_dict=grouping_dict
#     trace_grouping_dict=None
#     )

# #save_name = '/Users/cam/Downloads/ivs_clusters_movie_LSR_frame.html'
# save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/ivs_plots/ivs_clusters_movie_LSR_frame.html'


# fig3d = plot_3d.generate_3d_plot(
#     time = time_int,
#     show = False, 
#     save_name =save_name, 
#     static_traces = static_traces, 
#     static_traces_times = static_trace_times,
#     static_traces_legendonly=False,
#     #focus_group = 'Gamma Vel family (IVS)',
#     focus_group = None,
#     fade_in_time = 0, # Myr,
#     show_gc_line=False,
#     fade_in_and_out = False
# )

In [33]:
xyz_widths = (201, 201, 151)

plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    figure_title='Interactive Figure 8 from Gao et al. (2024) - Gamma Vel frame',
    xyz_widths = xyz_widths, 
    #xyz_ranges=xyz_ranges,
    figure_theme = 'light', 
    trace_grouping_dict=None)

#save_name = '/Users/cam/Downloads/ivs_clusters_movie_gvel_frame.html'
save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/ivs_plots/ivs_clusters_movie_gvel_frame_updated.html'


fig3d = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_trace_times,
    static_traces_legendonly=False,
    focus_group = 'Gamma Vel family',
    #focus_group = None,
    fade_in_time = 0, # Myr,
    show_gc_line=False,
    fade_in_and_out = False
)